In [1]:
import numpy as np
import matplotlib.tri as Tri
import matplotlib.pyplot as plt
from netCDF4 import Dataset as NetCDFFile 
import netCDF4
from netCDF4 import num2date, date2num, date2index
import datetime as dt
import pandas as pd
from StringIO import StringIO
import xarray as xr

/Users/bgh2-ra/anaconda2/lib/python2.7/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}
/Users/bgh2-ra/anaconda2/lib/python2.7/site-packages/distributed/config.py:20: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  defaults = yaml.load(f)


In [3]:
import re

def extractURLsatellite(fileURL, satName):
    """
    Function to extract the URLs for a specific satellite from the IMOS URLs list
    
    Parameters:
    -----------
    
    - fileURL : IMOS URLs list as a txt file
    - satName : name of the satellite such as JASON-2 JASON-3 
    
    Ouputs:
    ------
    
    - getFiles : list of URLs for the desired satellite
    
    """
    
    getFiles = []
    
    with open(fileURL) as f:
        for line in f:
            if re.search(r"%s"%satName, line):
                changeURL = re.sub('http://data.aodn.org.au', 'http://thredds.aodn.org.au/thredds/dodsC', line)
                getFiles.append(changeURL)
                
    return getFiles

In [4]:
jason2URL = extractURLsatellite( fileURL = 'IMOSURLs.txt', satName = 'JASON-2')

jason3URL = extractURLsatellite( fileURL = 'IMOSURLs.txt', satName = 'JASON-3')

saralURL = extractURLsatellite( fileURL = 'IMOSURLs.txt', satName = 'SARAL')

sentinel3aURL = extractURLsatellite( fileURL = 'IMOSURLs.txt', satName = 'SENTINEL-3A')

cryosat2URL = extractURLsatellite( fileURL = 'IMOSURLs.txt', satName = 'CRYOSAT-2')

enviURL = extractURLsatellite(fileURL = 'IMOSURLs.txt', satName = 'ENVISAT')

geosatURL = extractURLsatellite(fileURL = 'IMOSURLs.txt', satName = 'GEOSAT')

ersURL = extractURLsatellite(fileURL = 'IMOSURLs.txt', satName = 'ERS-2')

gfoURL = extractURLsatellite(fileURL = 'IMOSURLs.txt', satName = 'GFO')

topURL = extractURLsatellite(fileURL = 'IMOSURLs.txt', satName = 'TOPEX')

In [5]:
allURL = []
allURL.append(jason2URL)
allURL.append(jason3URL)
allURL.append(saralURL)
allURL.append(sentinel3aURL)
allURL.append(cryosat2URL)
allURL.append(enviURL)
allURL.append(geosatURL)
allURL.append(ersURL)
allURL.append(gfoURL)
allURL.append(topURL)

In [6]:
boxLat = []
boxLon = []
boxWh = []
boxT = []
boxQ = []

#Cape Melville coordinates
latmin = -14.0
latmax = -13.0  
lonmin = 145.0
lonmax = 146.0

if latmin>latmax:
    print('Error wrong definition of min and max lat!!!')

if lonmin>lonmax:
    print('Error wrong definition of min and max lon!!!')

In [7]:
#Choose start dates

start_date = dt.datetime(1985,1,1)
end_date = dt.datetime(2019,2,21)

In [8]:
for u in range (len(allURL)):
    urlON = allURL[u]
    for k in range (len(urlON)):
        ncs = NetCDFFile(urlON[k])
        lats = ncs.variables['LATITUDE'][:]
        lons = ncs.variables['LONGITUDE'][:]
       
        if u == 2:
            wh = ncs.variables['SWH_KA_CAL'][:]
            qc = ncs.variables['SWH_KA_quality_control'][:]
        
        else:
            wh = ncs.variables['SWH_KU_CAL'][:]
            qc = ncs.variables['SWH_KU_quality_control'][:]

        # Get desired time interval  
        time_var = ncs.variables['TIME']
        tt = ncs.variables['TIME'][:]
        
        timing = netCDF4.num2date(tt,time_var.units)
        
        #data in correct time, quality flag and lat/lon
        for k in range(len(timing)):
            if timing[k] >= start_date and timing[k] <= end_date:
                if lats[k]>latmin and lats[k]<latmax and lons[k]>lonmin and lons[k]<lonmax:
                    boxLat.append(lats[k])
                    boxLon.append(lons[k])
                    boxWh.append(wh[k])
                    boxT.append(tt[k])
                    boxQ.append(qc[k])

In [14]:
len(boxT)
boxT

36666

In [15]:
for k in range(len(boxLat)):
    if k == 0:
        lat = boxLat[k]
        lon = boxLon[k]
        wh = boxWh[k]
        tt = boxT[k]
        qc = boxQ[k]
    else:
        lat = np.append(lat,boxLat[k])
        lon = np.append(lon,boxLon[k])
        wh = np.append(wh,boxWh[k])
        tt = np.append(tt,boxT[k])
        qc = np.append(qc,boxQ[k])

21.49700164794922

Combine into individual satellite tracks

In [18]:
days = [] #daystart
daye = [] #dayend
for k in range(len(wh)):
    t1 = netCDF4.num2date(wh[k],time_var.units)
    if k == 0: #get day 0. Then else: every day after that
        days.append(0)
        dd = netCDF4.num2date(wh[k],time_var.units)
        it = 0
    else:
        if t1.day != dd.day: #if day 1 is not equal to day 2, then append
            #print dd.day,t1.day
            daye.append(k-1)
            days.append(k)
            it += 1
            dd = netCDF4.num2date(wh[k],time_var.units)

Get data with wave heights>0, quality flag =1

In [20]:
above = []

for k in range(len(days)):
    ids = np.where(np.logical_and(wh>0,qc==1))
    if k ==0:
        above.append(0)
    else:
        above.append(k)

In [41]:
ids = np.where(np.logical_and(wh>0,qc==1))

In [22]:
len(above) #This therefore is: mean value of each satellite track with above 0 and qc==1

6130

In [35]:
ss = wh[above]

In [36]:
len(ss)

6130

In [40]:
print max(ss)
print np.mean (ss)
print min(ss)

19.327001571655273
1.668615086828351
0.0


In [25]:
meanwh = [] #mean wave height from each satellite track

for t in range(len(days)-1):
    meanwh.append(np.mean(wh[days[t]:days[t+1]]))

2.0540000200271606